In [1]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
import custom_library as cl
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

In [2]:
enlace_drive = "https://drive.google.com/uc?id=1c1l3OMUFjQWcydV0srKe95LLEfkBMKia"

# Cargar el archivo CSV en un DataFrame de pandas
original_data = pd.read_csv(enlace_drive)
modified_data = original_data.copy()
columns_to_keep = ['label', 'acousticness', 'instrumentalness', 'key', 'mode', 'time_signature']

In [3]:
# Step 1: 
binarization = FunctionTransformer(lambda data: cl.binarize_variables(data))

# Step 2: 
removal = FunctionTransformer(lambda data: data.drop(columns=columns_to_keep))

# Step 3: Handle outliers and missing data
handle_outliers = FunctionTransformer(lambda data: cl.handle_outliers(data, method='Imputacion', imputation_method='MICE', winsorization_rate=None))
imputation = FunctionTransformer(lambda data: cl.compute_missing_data(data, method='KNN', testing=True, percentage=0.05))

# Step 4: Transformation of variables
normalization = FunctionTransformer(lambda data: cl.transform_data(data, method='yeo-johnson', p_thres=0.05))

# Step 5: Dimensionality reduction
dimensionality = FunctionTransformer(lambda data: cl.reduce_dimensionality(data, method='Projection', corr_thres=0.8, var_thres=0.01, normality_thres=0.05, explained_var=0.93, do_ica=False))

# Step 6: 
def add_columns_back(modified_data, original_data, columns_to_keep):
    modified_data = pd.DataFrame(modified_data, columns=[col for col in original_data.columns if col not in columns_to_keep])
    result = pd.concat([modified_data.reset_index(drop=True), original_data[columns_to_keep].reset_index(drop=True)], axis=1)
    return result
addition = FunctionTransformer(lambda data: add_columns_back(data, original_data, columns_to_keep), validate=False)

In [4]:
pipeline = Pipeline(steps=[
    ('binarization', binarization),
    ('removal', removal),
    ('handle_outliers', handle_outliers),
    ('imputation', imputation),
    ('normalization', normalization),
    ('addition', addition)
])

In [5]:
transformed_data = pipeline.fit_transform(original_data)
transformed_data.describe()

,danceability,duration,energy,liveness,loudness,speechiness,tempo,valence,label,acousticness,instrumentalness,key,mode,time_signature
count,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000
mean,-1.077656e-16,-2.806644e-16,-3.481659e-16,5.376440e-16,2.984279e-16,-2.190840e-16,-3.718507e-16,8.408089e-17,0.602667,0.357394,0.100245,4.829333,0.741333,3.902667
std,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,0.489673,0.338405,0.259921,3.636001,0.438194,0.400091
min,-2.456003e+00,-2.751513e+00,-1.983064e+00,-2.533777e+00,-2.173560e+00,-1.723473e+00,-2.678377e+00,-2.103318e+00,0.000000,0.000001,0.000000,0.000000,0.000000,1.000000
25%,-6.877006e-01,-6.467967e-01,-7.418552e-01,-7.570056e-01,-7.107680e-01,-8.646402e-01,-7.278264e-01,-8.285330e-01,0.000000,0.037150,0.000000,1.000000,0.000000,4.000000
50%,1.444718e-02,1.878049e-02,4.908488e-02,-1.506445e-01,4.245734e-02,-1.658211e-01,5.406492e-02,5.530534e-03,1.000000,0.244500,0.000010,5.000000,1.000000,4.000000
75%,6.473217e-01,5.726352e-01,8.144655e-01,7.538507e-01,7.282859e-01,7.703461e-01,6.956556e-01,7.894804e-01,1.000000,0.678500,0.002245,8.000000,1.000000,4.000000
max,2.593526e+00,2.700316e+00,1.863444e+00,2.033538e+00,3.401593e+00,1.987191e+00,2.454960e+00,1.947572e+00,1.000000,0.994000,0.967000,11.000000,1.000000,5.000000


In [6]:
model = RandomForestClassifier(max_depth=5,n_estimators=25)
x = transformed_data.drop(columns=['label'])
y = transformed_data['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model.fit(x_train, y_train) 
y_pred = model.predict(x_test) 
report = classification_report(y_test, y_pred, output_dict = True)
accuracy = report['accuracy']
print(classification_report(y_test, y_pred))
joblib.dump(model, 'random_forest_model.pkl')

              precision    recall  f1-score   support

           0       0.71      0.78      0.74        54
           1       0.87      0.82      0.84        96

    accuracy                           0.81       150
   macro avg       0.79      0.80      0.79       150
weighted avg       0.81      0.81      0.81       150



['random_forest_model.pkl']